In [2]:
import pandas as pd
import numpy as np
import os
import subprocess
import datetime
from pathlib import Path

In [20]:
def prep_gengraph_input_output(date, end_date, seq_names):
    date = date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    output_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output")
    run_path = output_path / f"{date}_{end_date}"
    
    if not run_path.exists():
        run_path.mkdir()
        
    seq_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston")
    seq_file_dest = run_path / f"{date}_{end_date}_seq_input.txt"
    with open(seq_file_dest, "w") as dest:
        dest.write("seq_name\taln_name\tseq_path\tannotation_path\n")
        for ix, seq in enumerate(seq_names):
            dest.write(f"{seq}\tseq_{ix}\t{seq_path / seq}.fasta\n")
            
    return run_path, seq_file_dest, f"{date}_{end_date}"

def run_pipeline(date, end_date, seq_names, exp_name):
    base_path = Path("/scratch/jho5ze/bionets/covid/")
    uncert_file = "uncert.csv"
    
    date = date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    output_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output")
    run_path = output_path / exp_name / f"{date}_{end_date}"
    uncert_file = run_path / "uncert.csv"
    
    
    if not run_path.exists():
        run_path.mkdir(parents=True)
        
    seq_path = Path("/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston")
    
    command = f"sbatch {base_path / 'scripts/run_pipeline.sbatch'} {run_path} {uncert_file}".split()
    command += [f"{seq_path / seq}.fasta" for seq in seq_names]
    print(command)
#     subprocess.run(command)
#     with open(seq_file_dest, "w") as dest:
#         dest.write("seq_name\taln_name\tseq_path\tannotation_path\n")
#         for ix, seq in enumerate(seq_names):
#             dest.write(f"{seq}\tseq_{ix}\t{seq_path / seq}.fasta\n")
            
#     return run_path, seq_file_dest, f"{date}_{end_date}"

In [23]:
data_path = Path("/scratch/jho5ze/bionets/covid/data")

seqs = pd.read_csv(data_path / "houston_metadata.csv", header=None, parse_dates=[1])
seqs.columns = ["seq", "date"]

date_window = 5
date_step_size = 5

for date in pd.date_range(seqs.date.min(), seqs.date.max(), freq=f"{date_step_size}D"):
    end_date = date + np.timedelta64(date_window - 1, 'D')
    sub_seqs = seqs[(seqs.date >= date) & (seqs.date <= end_date)].seq
#     output_directory, seq_file, out_file = prep_gengraph_input_output(date, end_date, sub_seqs)
    experiment_name = f"window_{date_window}_step_{date_step_size}"
    run_pipeline(date, end_date, sub_seqs, experiment_name)
#     break
#     if not out_file + ".xml" in os.listdir(output_directory):
#         print(out_file)
    
#         command = f"sbatch {base_path / 'scripts/run_gengraph.sbatch'} {output_directory} {seq_file} {out_file}"
#         subprocess.run(command.split())
#     break


['sbatch', '/scratch/jho5ze/bionets/covid/scripts/run_pipeline.sbatch', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output/window_5_step_5/2020-03-18_2020-03-22', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output/window_5_step_5/2020-03-18_2020-03-22/uncert.csv', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston/TX-HMH-793.fasta', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston/TX-HMH-792.fasta']
['sbatch', '/scratch/jho5ze/bionets/covid/scripts/run_pipeline.sbatch', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output/window_5_step_5/2020-03-23_2020-03-27', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output/window_5_step_5/2020-03-23_2020-03-27/uncert.csv', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston/TX-HMH-1761.fasta']
['sbatch', '/scratch/jho5ze/bionets/covid/scripts/run_pipeline.sbatch', '/sfs/lustre/bahamut/scratch/jho5ze/bionets/covid/data/Houston_output/windo

In [42]:
seqs.date.min()

Timestamp('2020-03-18 00:00:00')